In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

데이터 전처리  
우리가 실제로 접하게 되는 데이터는 사이킷런에서 제공하는 데이터와는 다르게 머신러닝을 적용하기 전에 데이터를 가공해야 하는 경우가 많다. 이를 데이터 전처리, 피처 엔지니어링 등으로 부른다.

결측치 처리  
데이터셋의 일부에 데이터가 존재하지 않을 경우 결측치(missing value)라고 부르며, 머신러닝 알고리즘을 적용하기 전에 미리 결측치를 처리하는 것이 중요하다.

In [3]:
df = pd.DataFrame([
    [42, 'male', 12, 'reading', 'class2'],
    [35, 'unknown', 3, 'cooking', 'class1'],
    [1000, 'female', 7, 'cycling', 'class3'],
    [1000, 'unknown', 21, 'unknown', 'unknown']
])
df.columns = ['age', 'gender', 'month_birth', 'hobby', 'target'] # 나이, 성별, 태어난 달, 취미, 타겟
df
# 데이터프레임에 데이터가 존재하지 않는 결측치는 없지만 부적절한 값이 존재한다.
# 나이에 1000이라는 값이 존재하는 것과 나머지 피쳐에 unknown이라는 값으로 입력된 부적절한 값은 결측치로 처리하는
# 것이 좋다.

,age,gender,month_birth,hobby,target
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


In [4]:
# unique() 함수로 각 열의 유니크한 값을 확인한다.
print('age: ', df['age'].unique()) # 인간의 수명을 고려할 때 1000이라는 값은 부적절한다.
print('gender: ', df['gender'].unique()) # male과 female을 제외한 값은 부적절하다.
print('month_birth: ', df['month_birth'].unique()) # 1년은 12월까지 존재하므로 21을 부적절하다.
print('hobby: ', df['hobby'].unique()) # 취미의 unknown은 부적절하다.
print('target: ', df['target'].unique()) # 타겟 변수도 unknown은 부적절하다.

age:  [  42   35 1000]
gender:  ['male' 'unknown' 'female']
month_birth:  [12  3  7 21]
hobby:  ['reading' 'cooking' 'cycling' 'unknown']
target:  ['class2' 'class1' 'class3' 'unknown']


In [5]:
# 부적절한 데이터값을 결측치로 바꾸기 위해서는 np.nan을 사용한다.
df.loc[df['age'] > 150, ['age']] = np.nan
df.loc[df['gender'] == 'unknown', ['gender']] = np.nan
df.loc[df['month_birth'] > 12, ['month_birth']] = np.nan
df.loc[df['hobby'] == 'unknown', ['hobby']] = np.nan
df.loc[df['target'] == 'unknown', ['target']] = np.nan
df

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


In [6]:
# 데이터셋 각 열의 결측치 개수를 확인한다.
df.isnull().sum() # df.isna().sum()

age            2
gender         2
month_birth    1
hobby          1
target         1
dtype: int64

결측치를 처리하는 데 주로 쓰이는 방법은 결측치를 삭제하거나 다른 값으로 변경하는 방법이 있다.

In [7]:
# dropna() 함수를 실행할 때 axis를 생략하거나 axis=0으로 옵션을 지정하면 결측치가 하나라도 포함된 행을 삭제한다.
df2 = df.dropna(axis=0)
df2

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2


In [8]:
# dropna() 함수를 실행할 때 axis=1로 옵션을 지정하면 결측치가 하나라도 포함된 열을 삭제한다.
df3 = df.dropna(axis=1)
df3

""
0
1
2
3


In [9]:
# dropna() 함수를 실행할 때 how='all' 옵션을 지정하면 행의 값이 모두 결측치인 행을 삭제한다.
df4 = df.dropna(how='all')
df4

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [10]:
# dropna() 함수를 실행할 때 thresh=n 옵션을 지정하면 결측치를 제외한 값의 개수가 n보다 작은 행을 삭제한다.
df5 = df.dropna(thresh=2) # 결측치를 제외한 값의 개수가 2개 미만인 행 삭제
df5

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [11]:
# dropna() 함수를 실행할 때 subset=['열이름'] 옵션을 사용하면 특정 열에 결측치가 존재할 때만 행을 삭제한다.
df6 = df.dropna(subset=['gender'])
df6

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [12]:
# fillna() 함수의 인수로 결측치를 대체할 값을 지정할 수 있다.
df7 = df.fillna(0)
df7

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,0,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,0,0.0,0,0


In [13]:
# fillna() 함수의 인수로 결측치를 대체할 값이 지정된 딕셔너리를 넘겨주면 열별로 결측치를 대체할 값을 지정할 수
# 있다.
# 각 열의 결측치를 대신할 값이 저장된 딕셔너리
alter_values = {'age': 0, 'gender': 'U', 'month_birth': 0, 'hobby': 'U', 'target': 'class4'}
df8 = df.fillna(value=alter_values)
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


클래스 라벨 설정

사이킷런 사용

In [15]:
# target 열을 보면 문자열(string) 형태인데, 데이터 전처리 과정에서 문자열을 정수(int)로 바꿔야 하는 경우가 있다.
from sklearn.preprocessing import LabelEncoder # 클래스 라벨링을 위해 import 한다.

In [16]:
df9 = df8
class_label = LabelEncoder() # 클래스 라벨링을 실행하는 객체를 생성한다.
data_value = df9['target'].values # 라벨링할 데이터를 얻어온다.
# print(type(data_value)) # <class 'numpy.ndarray'>
print(data_value)
# fit_transform() 함수로 라벨링할 데이터를 넘겨 라벨링을 실행한다.
y_new = class_label.fit_transform(data_value)
print(y_new)

['class2' 'class1' 'class3' 'class4']
[1 0 2 3]


In [17]:
# 라벨링한 데이터를 데이터프레임에 넣어준다.
df9['target'] = y_new
df9

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,1
1,35.0,U,3.0,cooking,0
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


In [18]:
# inverse_transform() 함수로 클래스 라벨링한 데이터를 원래대로 되돌릴 수 있다.
y_ori = class_label.inverse_transform(y_new)
print(y_ori)
df9['target'] = y_ori
df9

['class2' 'class1' 'class3' 'class4']


,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


사이킷런을 사용하지 않고 직접 라벨링하기

In [19]:
# 라벨링 할 타겟 데이터를 얻어와 오름차순으로 정렬한다.
y_arr = df9['target'].values
y_arr.sort()
print(y_arr)

['class1' 'class2' 'class3' 'class4']


In [20]:
# 반복문을 실행해서 정렬된 타겟 데이터를 key로 일련번호를 value로 하는 딕셔너리를 만든다.
num_y = 0
dic_y = {}
for ith_y in y_arr:
    dic_y[ith_y] = num_y
    num_y += 1
print(dic_y)

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}


In [21]:
# replace() 함수를 이용해 딕셔너리에 맵핑된 데이터로 타겟을 라벨링 한다.
df9['target'] = df9['target'].replace(dic_y)
df9

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,0
1,35.0,U,3.0,cooking,1
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


원-핫 인코딩(one-hot encoding)  
원-핫 인코딩은 클래스 라벨링의 또 다른 방법으로 오직 0과 1만 사용한 벡터를 이용해 데이터 값을 나타내는 것이다.

판다스 라이브러리를 이용한 원-핫 인코딩

In [22]:
df10 = df9
df10['target'] = df10['target'].astype(str) # 타겟 데이터 값을 문자열로 변환한다.
df10

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,0
1,35.0,U,3.0,cooking,1
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


In [23]:
# 판다스의 get_dummies() 함수의 인수로 타겟 변수를 넣어 원-핫 인코딩을 한다.
df11 = pd.get_dummies(df10['target'])
df11

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [24]:
# 앞선 방법에서는 길이가 4인 벡터를 이용해 클래스를 구분했다. 이번에는 벡터의 길이를 하나 줄인 벡터를 이용해
# 원-핫 인코딩을 한다.
df10['target'] = df10['target'].astype(str)
# get_dummies() 함수에 drop_first=True 옵션을 지정하면 3개의 0과 1의 조합으로만 4개의 클래스가 구분되는 것을
# 볼 수 있다.
df12 = pd.get_dummies(df10['target'], drop_first=True)
df12

,1,2,3
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,1


사이킷런 라이브러리를 이용한 원-핫 인코딩  
판다스는 원-핫 인코딩 결과를 데이터프레임 형식으로 출력하는 반면에 사이킷런 라이브러리의 OneHotEncoder를 사용하면 array 형식으로 출력한다.

In [33]:
from sklearn.preprocessing import OneHotEncoder
hot_encoder = OneHotEncoder() # OneHotEncoder 객체를 만든다.
y = df8[['target']]
# print(y)
# fit_transform() 함수로 원-핫 인코딩을 실행한다.
y_hot = hot_encoder.fit_transform(y)
print(y_hot)
print(y_hot.toarray())

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


텐서프로우 라이브러리를 이용한 원-핫 인코딩

In [35]:
from tensorflow.keras.utils import to_categorical
y_hotec = to_categorical(y)
print(y_hotec)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


데이터 스케일링(data scaling)  
피쳐는 제각기 다른 단위를 가지기 때문에 이로 인해 숫자 자체의 크기 차이가 발생한다. 데이터 스케일링은 데이터 값이 단위의 영향을 받지 않도록 변형하는 것을 의미한다.  
전체 데이터를 트레이닝 데이터와 테스트 데이터로 나누고 스케일링 기준이 되는 값을 구할 때는 트레이닝 데이터만으로 구한다.

표준화 스케일링(standard scaling)  
대표적인 데이터 스케일링 방법으로 데이터가 평균 0, 표준 편차 1일 되도록 변경하는 방법이다.

$$\frac{x_i - \bar x}{\sigma}$$

위 식의 의미는 기존 데이터 값에서 평균을 뺀 후 표준 편차로 나눈 값으로 바꾼다는 의미이다.

In [36]:
# 표준화 스케일링을 위한 StandardScaler 함수를 불러온다.
from sklearn.preprocessing import StandardScaler

In [41]:
std = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화 스케일러를 적용할 데이터를 반드시 fit() 함수로 먼저 적합 시킨 후 transform() 함수를 실행해야 한다.
std.fit(df8[['month_birth']]) # 표준화 스케일러에 month_birth 열을 적합시킨다.
# transform() 함수로 적합(fit)된 표준화 스케일러를 기준으로 month_birth 열 데이터 값을 변형시킨다.
x_std = std.transform(df8[['month_birth']])
print(x_std)

[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [42]:
# fit() 함수와 transform() 함수로 실행한 표준화 스케일링을 fit_transform() 함수로 한 번에 실행할 수 있다.
x_std2 = std.fit_transform(df8[['month_birth']])
print(x_std2)

[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [43]:
# 실제로 표준화 스케일링 이후 데이터가 평균 0, 표준 편차 1인지 확인한다.
print('평균: {:.1f}, 표준편차: {:.1f}'.format(np.mean(x_std), np.std(x_std)))

평균: -0.0, 표준편차: 1.0


로버스트 스케일링(robust scaling)  
로버스트 스케일링은 표준화 스케일링을 변형한 방법으로 중앙값(median)과 사분위수(quantile)을 사용한다. 이에 따라 극단값의 영향을 거의 받지 않는다는 장점이 있다.

$$\frac{x_i - q_2}{q_3 - q_1}$$

위 식에서 $q_1$은 1사분위수, $q_2$은 2사분위수(중위수), $q_3$은 3사분위수를 의미한다.

In [45]:
# 로버스트 스케일링을 위한 RobustScaler 함수를 불러온다.
from sklearn.preprocessing import RobustScaler

In [46]:
robust = RobustScaler() # 로버스트 스케일링 객체를 만든다.
robust.fit(df8[['month_birth']])
x_robust = robust.transform(df8[['month_birth']])
print(x_robust)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


In [47]:
x_robust2 = robust.fit_transform(df8[['month_birth']])
print(x_robust2)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


최소-최대 스케일링(min-max scaling)  
최소-최대 스케일링 방법은 데이터 값의 최대값이 1, 최소값 0으로 데이터 값이 가질 수 있는 범위를 제한한다.

$$\frac{x_i - min(x)}{max(x) - min(x)}$$

In [48]:
# 최소-최대 스케일링을 위한 MinMaxScaler 함수를 불러온다.
from sklearn.preprocessing import MinMaxScaler

In [50]:
minmax = MinMaxScaler() # 최소-최대 스케일링 객체를 만든다.
minmax.fit(df8[['month_birth']])
x_minmax = minmax.transform(df8[['month_birth']])
print(x_minmax)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


In [52]:
x_minmax2 = minmax.fit_transform(df8[['month_birth']])
print(x_minmax2)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


노멀(normalizer) 스케일링  
노멀 스테일링은 벡터의 유클라디안 거리가 1이 되도록 데이터 값을 변경한다.  
노멀 스케일링은 주로 벡터의 길이는 상관없고, 방향(각도)만 고려할 때 사용한다. 앞서 언급한 세 가지 스케일러는 열 기준인 것과 달리 노멀 스케일링은 행 기준이다.

$$\frac{x_i}{\sqrt{x_i^2 + y_i^2 + z_i^2}}$$

In [54]:
# 노멀 스케일링을 위한 Normalizer 함수를 불러온다.
from sklearn.preprocessing import Normalizer

In [57]:
normal = Normalizer() # 노멀 스케일링 객체를 만든다.
normal.fit(df8[['age', 'month_birth']])
x_normal = normal.transform(df8[['age', 'month_birth']])
print(x_normal)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


In [58]:
x_normal2 = normal.fit_transform(df8[['age', 'month_birth']])
print(x_normal2)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


데이터 스케일링 과정에서 fit() 함수는 트레이닝 데이터셋에 대해서만 사용하며, 테스트 데이터셋에는 fit() 함수를 사용하지 않고 transform() 함수만 사용한다. 그 이유는 데이터 스케일러는 트레이닝 데이터를 기반으로 만들기 때문이다.

from sklearn.preprocessing import StandardScaler

stand_scale = StandardScaler()  
x_train_std = stand_scale.fit_transform(x_train) # 트레이닝 데이터셋  
x_test_std = stand_scale.transform(x_test) # 테스트 데이터셋  

만약 stand_scale.transform(x_test)이 아닌 stand_scale.fit_transform(x_test)를 사용하면 테스트 데이터의 평균, 표준 편차를 이용해 스케일링을 적용하므로 앞서 트레이닝 데이터를 이용해 생성했던 스케일링 범위 및 파라미터가 달라진다.